# 1. 데이터 로드 및 전처리

In [ ]:
import pandas as pd
import altair as alt
from ipywidgets import interact, widgets
from vega_datasets import data

# Altair 데이터 변환 설정
alt.data_transformers.disable_max_rows()

# 데이터 로드
DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv')
MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv')
MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv')
PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv')
TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv')
TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv')
TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv')
TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv')

# 데이터 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# Parameter 컬럼 추가
all_data['Parameter'] = all_data['Elem'].map(param_map)

# 월 컬럼 정의
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [20]:
all_data.shape

(37283, 23)

In [21]:
# -99 값을 포함하는 행 삭제
all_data = all_data[~all_data.isin([-99]).any(axis=1)]

In [22]:
all_data.shape

(37283, 23)

# 2. Heatmap 출력

In [23]:
# Altair 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

def plot_heatmap(selected_param, selected_month):
    # 선택된 파라미터와 월 기준으로 데이터 필터링
    filtered = all_data[all_data['Parameter'] == selected_param]

    # Heatmap 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    heatmap = alt.Chart(filtered).mark_circle().encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        color=alt.Color(f'{selected_month}:Q', scale=alt.Scale(scheme='viridis')),
        size=alt.Size(f'{selected_month}:Q', title='Intensity'),
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return background + heatmap

# 3. UI 구성 (아이콘 및 파라미터 선택)

In [ ]:
# 파라미터 선택 위젯
param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Precipitation (mm)',
    description='Parameter:'
)

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jan',
    description='Month:'
)

# 필터링 슬라이더 추가
slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=500,
    step=1,
    description='Filter:',
    layout=widgets.Layout(width='80%')
)

def update(selected_param, selected_month):
    chart = plot_heatmap(selected_param, selected_month)
    chart.display()

interact(
    update,
    selected_param=param_dropdown,
    selected_month=month_dropdown
)


interactive(children=(Dropdown(description='Parameter:', options=('Precipitation (mm)', 'Number of Days with P…

<function __main__.update(selected_param, selected_month)>

In [7]:
def add_flags(selected_param, selected_month, value_range):
    # 데이터 필터링
    min_val, max_val = value_range
    filtered = all_data[
        (all_data['Parameter'] == selected_param) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    # Altair 지도 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    points = alt.Chart(filtered).mark_text(text="🚩", size=15, color='red').encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return background + points

In [ ]:
import pandas as pd
import altair as alt
from ipywidgets import interact, widgets
from vega_datasets import data

# Altair 데이터 변환 설정
alt.data_transformers.disable_max_rows()

# 데이터 로드
DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv')
MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv')
MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv')
PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv')
TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv')
TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv')
TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv')
TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv')

# 데이터 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# Parameter 컬럼 추가
all_data['Parameter'] = all_data['Elem'].map(param_map)

# 월 컬럼 정의
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Altair 세계 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

# Heatmap 생성 함수
def plot_heatmap(selected_param, selected_month):
    # 선택된 파라미터와 월 기준으로 데이터 필터링
    filtered = all_data[all_data['Parameter'] == selected_param]

    # Heatmap 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    heatmap = alt.Chart(filtered).mark_circle().encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        color=alt.Color(f'{selected_month}:Q', scale=alt.Scale(scheme='viridis'), title='Value'),
        size=alt.Size(f'{selected_month}:Q', title='Intensity'),
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title=selected_param)
        ]
    )

    return background + heatmap

# 조건에 따라 깃발 추가 함수
def add_flags(selected_param, selected_month, value_range):
    # 데이터 필터링
    min_val, max_val = value_range
    filtered = all_data[
        (all_data['Parameter'] == selected_param) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    points = alt.Chart(filtered).mark_text(text="🚩", size=15, color='red').encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return points

# 파라미터 선택 위젯
param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Precipitation (mm)',
    description='Parameter:'
)

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jan',
    description='Month:'
)

# 필터링 슬라이더
slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=500,
    step=1,
    description='Filter:',
    layout=widgets.Layout(width='80%')
)

# 업데이트 함수
def update(selected_param, selected_month, value_range):
    # Heatmap 생성
    heatmap_chart = plot_heatmap(selected_param, selected_month)

    # 깃발 추가 (필터링 조건 기반)
    flag_chart = add_flags(selected_param, selected_month, value_range)

    # 두 차트 결합
    chart = heatmap_chart + flag_chart
    chart.display()

# Interact 호출
interact(
    update,
    selected_param=param_dropdown,
    selected_month=month_dropdown,
    value_range=slider
)

interactive(children=(Dropdown(description='Parameter:', options=('Precipitation (mm)', 'Number of Days with P…

<function __main__.update(selected_param, selected_month, value_range)>

In [ ]:
import pandas as pd
import altair as alt
from ipywidgets import interact, widgets
from vega_datasets import data

# Altair 데이터 변환 설정
alt.data_transformers.disable_max_rows()

# 데이터 로드
DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv')
MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv')
MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv')
PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv')
TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv')
TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv')
TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv')
TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv')

# 데이터 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# Parameter 컬럼 추가
all_data['Parameter'] = all_data['Elem'].map(param_map)

# 월 컬럼 정의
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Altair 세계 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

# Heatmap 생성 함수
def plot_heatmap(selected_month):
    # Heatmap 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    return background

# 조건에 따라 깃발 추가 함수
def add_flags(selected_month, param_values):
    # 모든 조건을 만족하는 데이터 필터링
    filtered = all_data
    for param_name, value_range in param_values.items():
        min_val, max_val = value_range
        filtered = filtered[
            (filtered['Parameter'] == param_name) &
            (filtered[selected_month] >= min_val) &
            (filtered[selected_month] <= max_val)
        ]

    points = alt.Chart(filtered).mark_text(text="🚩", size=15, color='red').encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return points

# 슬라이더 초기화
slider_dict = {}
sliders = {}
for param_name in param_map.values():
    slider_dict[param_name] = widgets.FloatRangeSlider(
        value=[0, 100],
        min=0,
        max=500,
        step=1,
        description=param_name,
        layout=widgets.Layout(width='80%')
    )
    sliders[param_name] = slider_dict[param_name]

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jan',
    description='Month:'
)

# 업데이트 함수
def update(selected_month, **param_values):
    # Heatmap 생성
    heatmap_chart = plot_heatmap(selected_month)

    # 깃발 추가 (필터링 조건 기반)
    flag_chart = add_flags(selected_month, param_values)

    # 두 차트 결합
    chart = heatmap_chart + flag_chart
    chart.display()

# Interact 호출
interact(
    update,
    selected_month=month_dropdown,
    **{
        param_name: sliders[param_name]
        for param_name in param_map.values()
    }
)

interactive(children=(Dropdown(description='Month:', options=('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',…

<function __main__.update(selected_month, **param_values)>

In [18]:
import pandas as pd
import altair as alt
from ipywidgets import interact, widgets
from vega_datasets import data

# Altair 데이터 변환 설정
alt.data_transformers.disable_max_rows()

# 데이터 로드
DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv')
MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv')
MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv')
PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv')
TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv')
TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv')
TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv')
TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv')

# 데이터 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# Parameter 컬럼 추가
all_data['Parameter'] = all_data['Elem'].map(param_map)

# 월 컬럼 정의
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Altair 세계 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

# 슬라이더 범위를 업데이트하는 함수
def update_sliders(selected_month):
    slider_dict = {}
    fixed_zero_params = [
        'Precipitation',
        'Number of Days with Precipitation ≥ 1 mm',
        'Mean Sea Level Pressure',
        'Mean Vapor Pressure',
        'Total Number of Hours of Sunshine'
    ]

    for param_name in param_map.values():
        # 데이터 필터링
        filtered = all_data[all_data['Parameter'] == param_name]
        valid_values = filtered[selected_month].dropna()  # NaN 제거
        
        # 최소값과 최대값 계산
        if not valid_values.empty:
            min_value = valid_values.min()
            max_value = valid_values.max()
        else:
            min_value, max_value = 0, 0  # 값이 없는 경우 기본값
        
        # 특정 파라미터의 경우 최소값을 0으로 설정
        param_base_name = param_name.split(' (')[0]
        if param_base_name in fixed_zero_params:
            min_value = 0
        
        # 슬라이더 정보 저장
        slider_dict[param_name] = {
            "min": min_value,
            "max": max_value
        }

    return slider_dict

# Heatmap 생성 함수
def plot_heatmap(selected_month):
    # Heatmap 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    return background

# 조건에 따라 깃발 추가 함수
def add_flags(selected_month, param_values):
    # 모든 조건을 만족하는 데이터 필터링
    filtered = all_data
    for param_name, value_range in param_values.items():
        min_val, max_val = value_range
        filtered = filtered[
            (filtered['Parameter'] == param_name) &
            (filtered[selected_month] >= min_val) &
            (filtered[selected_month] <= max_val)
        ]

    points = alt.Chart(filtered).mark_text(text="🚩", size=15, color='red').encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return points

# 초기 월 설정
selected_month = 'Jan'

# 슬라이더 초기화
slider_dict = update_sliders(selected_month)
range_sliders = {}
for param_name, limits in slider_dict.items():
    range_sliders[param_name] = widgets.FloatRangeSlider(
        value=[limits["min"], limits["max"]],
        min=limits["min"],
        max=limits["max"],
        step=0.1,
        description=param_name,
        layout=widgets.Layout(width='80%')
    )

# 월 변경 시 슬라이더 업데이트
def on_month_change(change):
    global range_sliders
    new_month = change['new']
    slider_dict = update_sliders(new_month)
    
    # 슬라이더 값 업데이트
    for param_name, slider in range_sliders.items():
        slider.min = slider_dict[param_name]["min"]
        slider.max = slider_dict[param_name]["max"]
        slider.value = [slider_dict[param_name]["min"], slider_dict[param_name]["max"]]

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value=selected_month,
    description='Month:'
)

# 월 변경 이벤트 핸들러
month_dropdown.observe(on_month_change, names='value')

# 최종 업데이트 함수
def update(selected_month, **param_values):
    # Heatmap 생성
    heatmap_chart = plot_heatmap(selected_month)

    # 깃발 추가 (필터링 조건 기반)
    flag_chart = add_flags(selected_month, param_values)

    # 두 차트 결합
    chart = heatmap_chart + flag_chart
    chart.display()

# Interact 호출
interact(
    update,
    selected_month=month_dropdown,
    **{
        param_name: range_sliders[param_name]
        for param_name in param_map.values()
    }
)

interactive(children=(Dropdown(description='Month:', options=('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',…

<function __main__.update(selected_month, **param_values)>

In [31]:
import pandas as pd
import numpy as np

# 데이터 로드: 값 앞뒤 공백 제거를 위한 skipinitialspace=True 추가
DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv', skipinitialspace=True)
MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv', skipinitialspace=True)
MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv', skipinitialspace=True)
PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv', skipinitialspace=True)
TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv', skipinitialspace=True)
TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv', skipinitialspace=True)
TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv', skipinitialspace=True)
TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv', skipinitialspace=True)

# 데이터프레임 리스트
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]

# -99 값을 NaN으로 변환하는 함수
def replace_minus_99_with_nan(df):
    return df.replace(-99, np.nan)

# 모든 데이터프레임에 적용
datasets = [replace_minus_99_with_nan(df) for df in datasets]

# 각 데이터프레임 업데이트
DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN = datasets

DP01

,Elem,Rgn,ID,WIGOS_ID,Latitude,Longitude,Elevation,Country,Station,Jan,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,2,1,60620,0-20000-0-60620,27.837,-0.194,279.2,Algeria,Adrar,0.4,...,0.3,0.2,0.1,0.1,0.3,0.5,0.4,0.4,0.1,3.4
1,2,1,60560,0-20000-0-60560,32.752,-0.594,1076.0,Algeria,AinSefra,2.8,...,2.5,2.1,1.4,1.1,2.9,3.7,3.2,2.9,2.3,30.4
2,2,1,60390,0-20000-0-60390,36.690,3.217,25.0,Algeria,AlgerDarElBeida,8.4,...,6.1,4.3,1.4,0.4,1.4,4.0,5.4,9.2,8.2,64.5
3,2,1,60360,0-20000-0-60360,36.822,7.803,3.0,Algeria,Annaba,10.3,...,7.0,4.7,2.0,0.6,1.8,6.2,7.2,10.0,10.8,79.0
4,2,1,60468,0-20000-0-60468,35.761,6.320,821.3,Algeria,Batna,5.1,...,5.2,5.1,2.2,1.7,3.0,4.6,4.2,4.3,5.2,50.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5392,2,6,3488,NaN,52.949,1.122,21.0,United_Kingdom,Weybourne,11.3,...,8.5,7.9,8.9,9.1,9.2,9.0,11.2,12.4,12.7,119.6
5393,2,6,3075,NaN,58.454,-3.090,36.0,United_Kingdom,WickAirport,16.5,...,11.8,11.7,11.1,11.8,12.5,12.7,16.6,17.5,16.0,165.6
5394,2,6,3866,NaN,50.576,-1.297,20.0,United_Kingdom,WightStCatherinesPoint,12.5,...,8.5,7.5,7.0,6.6,7.8,7.9,11.8,13.5,13.2,115.5
5395,2,6,3462,NaN,52.612,-0.468,74.0,United_Kingdom,Wittering,10.1,...,8.8,8.4,9.0,9.1,9.2,8.3,10.2,11.2,10.7,113.0


In [ ]:
# 데이터 전처리
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

# 모든 데이터프레임의 컬럼 이름도 공백 제거
def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# Parameter 컬럼 추가
all_data['Parameter'] = all_data['Elem'].map(param_map)

# 월 컬럼 정의
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
# Altair 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

def plot_heatmap(selected_param, selected_month):
    # 선택된 파라미터와 월 기준으로 데이터 필터링
    filtered = all_data[all_data['Parameter'] == selected_param]

    # Heatmap 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    heatmap = alt.Chart(filtered).mark_circle().encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        color=alt.Color(f'{selected_month}:Q', scale=alt.Scale(scheme='viridis')),
        size=alt.Size(f'{selected_month}:Q', title='Intensity'),
        tooltip=[
            'Station:N',
            'Country:N',
            alt.Tooltip(f'{selected_month}:Q', title='Value')
        ]
    )

    return background + heatmap

# 파라미터 선택 위젯
param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Precipitation (mm)',
    description='Parameter:'
)

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jan',
    description='Month:'
)

# 필터링 슬라이더 추가
slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=500,
    step=1,
    description='Filter:',
    layout=widgets.Layout(width='80%')
)

def update(selected_param, selected_month):
    chart = plot_heatmap(selected_param, selected_month)
    chart.display()

interact(
    update,
    selected_param=param_dropdown,
    selected_month=month_dropdown
)

interactive(children=(Dropdown(description='Parameter:', options=('Precipitation (mm)', 'Number of Days with P…

<function __main__.update(selected_param, selected_month)>